Tugas Konduksi Paksa Aliran Eksteral
Nama    : Shabri Ash Shiddieqy
NIM     : 13320075

Untuk dapat menentukan kalor yang terbuang maka memerlukan nilai Ts. Untuk mencari nilai Ts ini diperlukan iterasi tertutup dengan alur kerja sebagai berikut

<p align="center">
 <img width="180" src="Flowchart.png" alt="Flowchart">
</p>

Deklarasi nilai nilai variabel yang diketahui

In [8]:
TiC = 180+7+5       # Temperatur steam (Celcius)
TeC = 30+5          # Temperatur luar (Celcius)
CK = 273.15         # Kompensasi untuk konversi Celcius ke kelvin
Ti = TiC + CK       # Temperatur steam (Kelvin)
Te = TeC + CK       # Temperatur luar (Kelvin)
Velocity = 2        # kecepatan aliran paksa (m/s)
width = 10.31e-3    # ketebalan dinding pipa (m)
D2 = 323.8e-3       # diameter pipa (m)
D1 = D2-2*width     # diameter dalam pipa (m)
r1 = D1/2           # jari-jari dalam pipa (m)
r2 = D2/2           # jari-jari pipa (m)
P = 101325          # Tekanan udara (Pa)

Import library

In [9]:
import CoolProp.CoolProp as cp  # Coolprop untuk mendapatkan karakteristik fluida pada titik tertentu
import numpy as np              # Numpy untuk regresi

Deklarasi persamaan umum sebagai bentuk fungsi

In [10]:
def fRe(v, Velocity, D):        #Fungsi untuk menghitung bilagan reynolds
    return (D * Velocity) / v

def fNu(Re, Pr):                #Fungsi untuk menghitung bilangan Nusselt
    return 0.3 + (0.62 * Re**0.5 * Pr**(1/3))/((1+(0.4/Pr)**(2/3))**0.25) * (1+(Re/282000)**(5/8))**(4/5)

def Pr(Cp, k, mu):              #Fungsi untuk menghitung bilangan Prandtl
    return Cp * mu / k

def fh(Nu, k, D):               #Fungsi untuk menghitung koefisien konveksi
    return Nu * k / D

def fRc(r1, r2, k):             #Fungsi untuk menghitung resistansi konduksi
    return np.log(r2/r1) / (2 * np.pi * k)

def fRh(r, h):                 #Fungsi untuk menghitung resistansi konveksi
    return 1 / (2 * np.pi * h * (r))



Tanpa insualasi

In [11]:
#Regresi linear untuk menentukan koefisien konduksi
T_tabel = [400 , 600, 800, 1000]
k_tabel = [49.8 , 44.0 , 37.4 , 29.3]
k_pipe = np.interp(Ti, T_tabel, k_tabel)


#Menentukan resistansi konduksi pipa
Rk = fRc(r1,r2,k_pipe)

#Resistansi konveksi dipengaruhi Ts yang masih belum diketahui
#Oleh karena itu dibuat loop untuk mendekati nilai Ts hingga errornya lebih kecil dari 1%

#inisialisasi Ts
Ts = Te + 1
#Membuat loop untuk mendekati nilai Ts
error = 1
error1 = 1
i = 0
while error > 0.004:
    error = error1
    Tf = (Ts + Te)/2
    #Karakteristik fluida pada Tf 
    Pr = cp.PropsSI('Prandtl', 'T', Tf, 'P', 101325, 'Air') # Menghitung bilangan Prandtl dengan menggunakan Library CoolProp
    mu = cp.PropsSI('V', 'T', Tf, 'P', 101325, 'Air')       # Menghitung viskositas dengan menggunakan Library CoolProp
    rho = cp.PropsSI('D', 'T', Tf, 'P', 101325, 'Air')      # Menghitung densitas dengan menggunakan Library CoolProp
    k = cp.PropsSI('L', 'T', Tf, 'P', 101325, 'Air') 
    v = mu / rho
    Re = fRe(v, Velocity, D2)
    Nu = fNu(Re, Pr)
    h = fh(Nu, k, D2)
    Rh = fRh(r2, h)
    Q = (Ti-Te)/(Rk+Rh)
    Ts1 = Ts + Q*Rk
    error1 = abs(Ts1-Ts)/Ts1
    i+=1
print ("Besar heat loss adalah",Q,"W/m\nDengan jumlah iterasi",i)

Besar heat loss adalah 1566.9043073217172 W/m
Dengan jumlah iterasi 2
1566.9043073217172


Dengan insulasi

In [12]:
#Informasi lapisan insulasi
# Informasi insulator
widthI = 1*0.0254 # Ketebalan insulasi (m)
KI = 0.03
r3 = r2 + widthI # jari-jari insulator (m)

# Informasi Cladding
widthC = 2*0.0254 # Ketebalan cladding (m)
Kcladd = 52
r4 = r3 + widthC # jari-jari cladding (m)

# Diameter Pipa keseluruhan (m)
D3 = 2*r4           
RI = fRc(r2,r3,KI)
Rcaddl = fRc(r3,r3,Kcladd)

In [14]:
#Dilakukan iterasi kembali
Ts = Te + 1
#Membuat loop untuk mendekati nilai Ts
error = 1
error1 = 1
i = 0
while error > 0.001:
    error = error1
    Tf = (Ts + Te)/2
    #Karakteristik fluida pada Tf 
    Pr = cp.PropsSI('Prandtl', 'T', Tf, 'P', 101325, 'Air') # Menghitung bilangan Prandtl dengan menggunakan Library CoolProp
    mu = cp.PropsSI('V', 'T', Tf, 'P', 101325, 'Air')       # Menghitung viskositas dengan menggunakan Library CoolProp
    rho = cp.PropsSI('D', 'T', Tf, 'P', 101325, 'Air')      # Menghitung densitas dengan menggunakan Library CoolProp
    k = cp.PropsSI('L', 'T', Tf, 'P', 101325, 'Air') 
    v = mu / rho
    Re = fRe(v, Velocity, D3)
    Nu = fNu(Re, Pr)
    h = fh(Nu, k, D3)
    Rh = fRh(r4, h)
    Q = (Ti-Te)/(Rk+Rh+RI+Rcaddl)
    Ts1 = Ts + Q*Rk
    error1 = abs(Ts1-Ts)/Ts1
    i+=1


print ("Besar heat loss adalah",Q,"W/m\nDengan jumlah iterasi",i)

Besar heat loss adalah 184.2720158408624 W/m
Dengan jumlah iterasi 2


File dari kode ini dapat diakses pada [Github](https://github.com/Shiddieqy/KonduksiPaksa)